In [1]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import matplotlib.pyplot as plt
import torch.utils.data as data
import torchvision as tv
import torchvision.transforms as tf
import torchvision.models as mod
import torch.distributions as distributions
from PIL import Image
%matplotlib inline

# 1.概率分布包：torch.distributions

概率分布包，torch.distributions，包括可参数化的概率分布和采样函数.

这将允许为优化问题构造随机计算图(stochastic computation graphs)和随机梯度估计(stochastic gradient estimators).

这个包大体上遵循 TensorFlow Distributions 的设计.

不可能经过随机样本直接进行反向传播. 但是有两个主要办法通过创建代理函数，可以使得反向传播通过.

这就是：得分函数估计（score funciton estimator）、似然比估计（likehood ratio estimator）、增强和路径导数估计（REINFORCE and pathwise derivative estimator）.

作为增强学习中策略梯度方法的基础，增强（REINFORCE）在增强学习会经常被看到. 同时，路径导数估计作为变分自动编码器中重参数化的技巧，也在其中经常使用.

同时，得分函数仅需要样本函数的值；路径导数需要函数的导数.

下面在增强学习示例中讨论 得分函数 和 路径导数.

## 1.1 得分函数 Score Function

当概率密度函数关于它的参数可微时，仅需要sample()和log_prog()就可实现REINFORCE：

$\Delta \theta=\alpha r \frac{\partial \log p( a\mid \pi^{\theta}(s))}{\partial \theta}$

其中,$\theta$表示全部参数，$\alpha$表示学习率, $r$表示回报.

$p( a\mid \pi^{\theta}(s))$表示在状态s时给定策略$\pi^{\theta}$，采取行动a的概率.

在实际中，我们会从网络的输出中采样行动，把这个行动作用于一个环境，然后使用log_prob来构造等价的损失函数.

需要注意，我们使用一个负号，因为优化器使用梯度下降，而上述的假设是梯度上升.

通过使用分类策略，实现REINFORCE的代码如下：

见:https://pytorch.org/docs/stable/distributions.html#score-function

## 1.2 路径导数 Pathwise Derivative

另一种实现这种随机梯度或策略梯度的方法，是通过resample()方法，使用重参数化技巧. 因为在resample()方法中，参数化的随机变量，可以通过无参数随机变量的参数化确定性函数来构造.(大概是通过简单抽样来获得复杂的抽样).重参数的样本因此变得可微分.

代码示例见:https://pytorch.org/docs/stable/distributions.html#pathwise-derivative

# 2.分布函数类

## 2.1 所有概率分布的抽象基类：class torch.distributions.distribution.Distribution

见：https://pytorch.org/docs/stable/distributions.html#torch.distributions.distribution.Distribution

## 2.2 指数族分布的抽象基类：class torch.distributions.exp_family.ExponentialFamily

见：https://pytorch.org/docs/stable/distributions.html#torch.distributions.exp_family.ExponentialFamily

## 2.3 bernoulli分布

以指数族分布为基类.

创建一个以probs或logits为参数的bernoulli分布: 样本为0或者1，以probs概率采样到1 ，以1-probs概率采样到0.


即概率分布为：

$p(x \mid u) = u^{x}(1-u)^{1-x}, x=0或1, 0 \le u \le 1 且p(x = 1) = u.$

__它的广义形式为 范畴分布 Categorical Distribution，即广义的bernoulli分布.__

In [5]:
##构造p=0.5的bernoulli分布
bern=distributions.bernoulli.Bernoulli(0.5)

In [10]:
##单个采样
bern.sample()

tensor(0.)

In [19]:
##批采样
bern.sample((5,))

tensor([1., 1., 1., 1., 0.])

In [12]:
##均值
bern.mean

tensor(0.5000)

In [13]:
##标准差
bern.stddev

tensor(0.5000)

In [15]:
##方差
bern.variance

tensor(0.2500)

In [17]:
##熵：平均信息量
bern.entropy()

tensor(0.6931)

In [20]:
bern.param_shape

torch.Size([])

## 2.4 beta分布

以指数族为基类.

以concentration1($\alpha$，第一个参数) 和 concentration0($\beta$，第二个参数)为参数构建.

In [21]:
beta=distributions.beta.Beta(0.5,0.5)

In [22]:
beta.sample()

tensor(0.9007)

In [24]:
beta.sample((2,3))

tensor([[0.1234, 0.7893, 0.4707],
        [0.8890, 0.0053, 0.2972]])

In [25]:
beta.mean

tensor(0.5000)

In [26]:
beta.stddev

tensor(0.3536)

In [28]:
beta.variance

tensor(0.1250)

In [30]:
beta.entropy()

tensor(-0.2416)

In [31]:
beta.concentration0

tensor(0.5000)

In [32]:
beta.concentration1

tensor(0.5000)

## 2.5 二项分布


以Distribution为基类.

- 第一个参数为bernoulli试验次数
- 第二个参数为probs
- 第三个参数为logits


它的概率分布为：

$p(x \mid N, u) = C_{N}^{k}p^{k}(1-p)^{N-k}$

__它的广义形式为 多项分布 Multinomial Distribution.__

In [33]:
binom=distributions.binomial.Binomial(10,0.3)
binom.sample((5,))

tensor([4., 2., 0., 3., 4.])

In [39]:
binom.mean

tensor(3.)

In [40]:
binom.variance

tensor(2.1000)

In [6]:
binom2=distributions.binomial.Binomial(10,torch.tensor([0.2,0.6]))##两个相互独立的试验，分布进行10次
binom2.sample()

tensor([0., 6.])

In [3]:
## Multinomial Distribution 多项分布，具体见2.19.
multn=distributions.multinomial.Multinomial(10,torch.tensor([2.4,6.5,0.3]))##内部归一化
multn.sample((5,))

tensor([[1., 9., 0.],
        [3., 7., 0.],
        [4., 6., 0.],
        [2., 7., 1.],
        [3., 7., 0.]])

## 2.6 范畴分布 Categorical Distribution，又称为广义 bernoulli 分布



__它是bernoulli分布的推广.__

$x={x_{1},x_{2},...,x_{k}}$, x的分量采用one-of-k的方式（即2.22的OneHot形式）表示：

且$x_{k}=0或1,\sum_{k}^{K}x_{k}=1$，即x的分量中只有一个为1，其他都为0. 且$p(x_{k}) = u_{k},u_{k}\ge 0, \sum_{k}^{K}u_{k}=1$

则  __范畴分布 categorical distribution的概率分布为：__


$p(x \mid u)=\prod_{k}^{K}u_{k}^{x_{k}}$

__注意：输入的概率张量，不要求和为1，只要求非负. 它内部会归一化处理__

In [4]:
cat=distributions.categorical.Categorical(torch.tensor([0.3,1,0.6]))##内部进行归一化
cat.sample((10,))

tensor([1, 2, 0, 1, 0, 1, 0, 0, 2, 2])

In [53]:
cat.mean

tensor(nan)

In [54]:
cat.stddev

tensor(nan)

## 2.7 柯西分布 Cauchy Distribution

In [55]:
cauchy=distributions.cauchy.Cauchy(0,1)
cauchy.sample((3,))

tensor([-12.2510,  -3.6473,   0.4213])

In [56]:
##累积分布函数
cauchy.cdf(0)

tensor(0.5000)

In [57]:
cauchy.mean

tensor(nan)

## 2.8 Chi2分布

见：https://pytorch.org/docs/stable/distributions.html#torch.distributions.chi2.Chi2

## 2.9 迪利克雷分布 Dirichlet Distribution

见：https://pytorch.org/docs/stable/distributions.html#torch.distributions.dirichlet.Dirichlet

## 2.10 指数分布 Exponential Distribution


In [59]:
expo=distributions.exponential.Exponential(0.2)
expo.mean

tensor(5.)

In [60]:
expo.sample((3,))

tensor([3.5006, 1.1961, 9.9056])

In [61]:
expo.rsample((2,))

tensor([0.7276, 7.8601])

In [62]:
expo.cdf(5)

tensor(0.6321)

## 2.11 Fisher-Snedecor 分布

## 2.12 Gamma分布

- 第一个参数为concentration，即$\alpha$

- 第二个参数为比率，即$\beta$

In [63]:
gamma=distributions.gamma.Gamma(1,1)
gamma.sample((3,))

tensor([1.2861, 0.6661, 0.6356])

## 2.13 几何分布 直到第k次成功的概率

返回的样本值为[0,inf)

In [65]:
geo=distributions.geometric.Geometric(0.7)
geo.sample()

tensor(2.)

## 2.14 Gumbel 分布

见：https://pytorch.org/docs/stable/distributions.html#torch.distributions.gumbel.Gumbel

## 2.15 HalfCauchy 分布

见：https://pytorch.org/docs/stable/distributions.html#torch.distributions.half_cauchy.HalfCauchy

## 2.16 半正太分布 HalfNormal Distribution

X ~ Normal(0, scale)

Y = |X| ~ HalfNormal(scale)

In [68]:
hNorm=distributions.half_normal.HalfNormal(0.5)
hNorm.sample((3,))

tensor([0.2858, 0.2493, 0.3281])

## 2.17 拉普拉斯分布 Laplace Distribution

In [69]:
lap=distributions.laplace.Laplace(0,0.5)
lap.sample((3,))

tensor([-0.0419, -0.2101,  0.0024])

In [70]:
lap.mean

tensor(0.)

In [71]:
lap.variance

tensor(0.5000)

In [72]:
lap.cdf(0)

tensor(0.5000)

## 2.18 对数正太分布 LogNormal Distribution

In [73]:
logNormal=distributions.log_normal.LogNormal(0,2)
logNormal.sample((3,))

tensor([0.3765, 0.0614, 0.0091])

In [74]:
logNormal.mean

tensor(7.3891)

## 2.19 多项分布 Multinomial Distribution


__它是二项分布 binominal distribution的推广.__



$x={x_{1},x_{2},...,x_{k}}$, x的分量采用one-of-k的方式（即2.22的OneHot形式）表示：

且$x_{k}=0或1,\sum_{k}^{K}x_{k}=1$，即x的分量中只有一个为1，其他都为0. 且$p(x_{k}) = u_{k},u_{k}\ge 0, \sum_{k}^{K}u_{k}=1$

则：
$p(x \mid u)=\prod_{k}^{K}u_{k}^{x_{k}}$，这就是  __范畴分布 categorical distribution的概率分布.__

那么N个样本的数据集：$x_{1},x_{2},..x_{n}$的似然函数为：

$p(D \mid u) = \prod_{k}^{K}u_{k}^{m_{k}}，其中m_{k}=\sum_{n}^{N}x_{nk}$


那么Multinomial Distribution定义为：

$Mult(m_{1},m_{2},...,m_{k} \mid N,u) = \frac{N!}{m_{1}!m_{2}!...m_{k}!}\prod_{k}^{K}u_{k}^{m_{k}}$，这是  __多项分布 multinomial distribution的概率分布.__

其中，$\sum_{k}^{K}m_{k}=N$

这个和上面的范畴分布区别，这里需要给出试验次数为参数. 
对比 2.6


__注意：输入的概率张量，不要求和为1，只要求非负. 它内部会归一化处理__

In [87]:
multn=distributions.multinomial.Multinomial(10,torch.tensor([2.4,6.5,0.3]))
multn.sample((5,))

tensor([[3., 7., 0.],
        [3., 7., 0.],
        [1., 7., 2.],
        [1., 8., 1.],
        [1., 9., 0.]])

## 2.20 多变量的正太分布 MultivariateNormal Distribution

输入参数为均值向量与协方差矩阵.

In [89]:
mean=torch.zeros(2)
std=torch.eye(2)
mn=distributions.multivariate_normal.MultivariateNormal(mean,std)

In [90]:
mn.sample((3,))

tensor([[-1.1859, -0.4758],
        [-1.2784, -0.6580],
        [-1.6796, -0.1345]])

## 2.21 正太分布 Normal Distribution

In [91]:
norm=distributions.normal.Normal(0,1)
norm.sample((3,))

tensor([-1.4638,  0.8199, -0.9868])

## 2.22 OneHot Categorical 分布

生成 one-of-k 样本的分布

__注意：输入的概率张量，不要求和为1，只要求非负. 它内部会归一化处理__

In [92]:
oh=distributions.one_hot_categorical.OneHotCategorical(torch.tensor([0.3,0.4,0.2,0.1]))
oh.sample((3,))

tensor([[1., 0., 0., 0.],
        [0., 0., 0., 1.],
        [0., 1., 0., 0.]])

## 2.23 帕累托分布 Pareto Distribution

见： https://pytorch.org/docs/stable/distributions.html#torch.distributions.pareto.Pareto

## 2.24 泊松分布 Possion Distribution

In [93]:
poi=distributions.poisson.Poisson(2)
poi.mean

tensor(2.)

In [94]:
poi.variance

tensor(2.)

In [95]:
poi.sample((3,))

tensor([0., 1., 4.])

## 2.25 RelaxedBernoulli 分布

见：https://pytorch.org/docs/stable/distributions.html#torch.distributions.relaxed_bernoulli.RelaxedBernoulli

## 2.26 RelaxedOneHotCategorical

见：https://pytorch.org/docs/stable/distributions.html#torch.distributions.relaxed_categorical.RelaxedOneHotCategorical

## 2.27 t分布

- 第一个参数df为自由度

In [96]:
td=distributions.studentT.StudentT(3)
td.sample((3,))

tensor([2.4505, 1.8551, 0.7435])

## 2.28 分布变换

即对基本分布施加特定的变换，生成新的分布.

以下分布都是经过变换生成的新的分布：

- Gumbel

- HalfCauchy

- HalfNormal

- LogNormal

- Pareto

- RelaxedBernoulli

- RelaxedOneHotCategorial

### 示例：对均匀分布施加sigmoid变换+仿射变换，形成Logistic分布

In [98]:
base_distribution=distributions.uniform.Uniform(0,1)
transfroms=[distributions.transforms.SigmoidTransform().inv,distributions.transforms.AffineTransform(2,3)]
logistic=distributions.transformed_distribution.TransformedDistribution(base_distribution,transfroms)


In [99]:
logistic.sample((3,))

tensor([1.7891, 7.8242, 1.8971])

## 2.29 均匀分布

略.

# 3. KL 散度


## 3.1 计算两个分布的KL散度
计算两个分布的KL散度，即：

$KL(p\mid \mid q) = \int p(x) \log \frac{p(x)}{q(x)}$

输入的p和q，需要为Distribution对象.

In [100]:
##计算指数分布与正太分布的KL散度
expon=distributions.exponential.Exponential(0.5)
norm=distributions.normal.Normal(0,1)
distributions.kl.kl_divergence(expon,norm)

tensor(3.2258)

In [101]:
distributions.kl.kl_divergence(norm,expon)

tensor(inf)

In [102]:
distributions.kl.kl_divergence(norm,norm)

tensor(0.)

## 3.2 注册KL散度计算的装饰器

见：https://pytorch.org/docs/stable/distributions.html#torch.distributions.kl.register_kl

# 4. 变换

- transforms.Transform ：变换的抽象基类


- transforms.ComposeTransform：组合变换类


- transforms.ExpTransform：指数变换类


- transforms.PowerTransform：幂变换类


- transforms.SigmoidTransform：Sigmoid变换类


- transforms.AbsTransform： 绝对值变换类


- transforms.AffineTransform：仿射变换类


- transforms.SoftmaxTransform：softmax变换类


- transforms.StickBreakingTransform：


- transforms.LowerCholeskyTransform

# 5. 约束 Constraints

constraints.Constraint: 约束对象的抽象基类. 约束对象表示了一片区域，在这片区域上变量是有效的，因而可以进行优化.

见：https://pytorch.org/docs/stable/distributions.html#module-torch.distributions.constraints

# 6. 约束注册表 Constraint Registry

Pytorch提供了两个链接Constraint对象和Transform对象的Constraint Registry 对象.

这些对象均要求constraints输入，并返回 trnaforms，但是在双射性方面有不同的保证.

见：https://pytorch.org/docs/stable/distributions.html#module-torch.distributions.constraint_registry